In [1]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn, topnFair

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
#ratings = pd.read_csv('/Users/denisehansen/Desktop/ITU/Thesis/Fair-Recommendations/ml-latest-small/ratings.1.csv', sep=',',
#                      names=['user', 'item', 'rating', 'timestamp'], header=0)

#items = pd.read_csv('/Users/denisehansen/Desktop/ITU/Thesis/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
#                      names=['movieId','title','genres'], header=0)

ratings = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/ratings.1.csv', sep=',',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)

items = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
                      names=['item','title','genres'], header=0)

#C:\Users\josse\Git-kode projects\Speciale\Fair-Recommendations\ml-latest-small\ratings.csv

In [4]:
#items

In [5]:
items_dummy = pd.concat([items, items['genres'].str.get_dummies(sep='|')], axis=1)
items_dummy.head()

,item,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [7]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [8]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 1, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    

test _reccomend
test _reccomend


In [9]:
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs_joined = all_recs.join(items_dummy.set_index('item'), on='item')
all_recs_joined.head()

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1961,3.416667,1,1,ItemItem,Rain Man (1988),Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1022,3.416667,1,2,ItemItem,Cinderella (1950),Animation|Children|Fantasy|Musical|Romance,0,0,0,...,0,0,0,1,0,1,0,0,0,0
2,296,3.416667,1,3,ItemItem,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2918,3.166667,1,4,ItemItem,Ferris Bueller's Day Off (1986),Comedy,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1580,3.166667,1,5,ItemItem,Men in Black (a.k.a. MIB) (1997),Action|Comedy|Sci-Fi,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [10]:
test_data = pd.concat(test_data, ignore_index=True)
test_data.head()

,user,item,rating
0,1,1953,4.0
1,1,1061,3.0
2,1,1343,2.0
3,1,2105,4.0
4,2,222,5.0


In [11]:
#all_recs_joined['item'].values

In [12]:
_protected_group2 = all_recs_joined.loc[all_recs_joined['Action'] == 1]
_protected_group2.head()
_protected_group2['item'].values

array([ 1580,  1210,   367,  5349,  1580,  1210,  2193,   367,  2193,
        5349,   377,  1580,  1210,   367,  2193,  2616,   592,  1918,
        3208,   589,  1372,   173,  1377,  2986,  1544,  2002,   380,
        1197,  2959,  8636,    10,   153,  2094,  2409,  2105,  2403,
        1356,  1374,  1917,   464,  3104,   434,  2916,   442,   110,
        2616,  3208,  1918,  2002,  2986,   380,  1544,  1377,  1372,
         173,  2916,  2409,  1374,  1917,  1356,  2403,  2105,  3104,
        2094,   434,   442,   464,  2115,  1291,  2194,  2991,  1387,
         112,   260,   349,  2000,  2005,  2628,  2640,  2716,  1208,
        2005,  1198,  2406,  2000,  2115,  2194,  1387,  1291,  1196,
        1396,  1200,   112,   260,   349,  3265,  1036,  2991,  1127,
         589,  2959,  1197,  2028,   405,   370,  2683,  2273,  8644,
        6502, 33679, 41569,   168,   552,   485,   208,   165,  8636,
       44191,   589,   592,  2194,  1396,  1387,  2406,  2628,  2640,
        1036,  1196,

In [13]:
# user , algorithms = iput to group_cols --> de koloner der skal  groupes på 
rla = topnFair.FairRecListAnalysis(['user', 'Algorithm'])
rla.add_metric("rND")
rla.add_metric("rKL")
rla.add_metric("rRD")
results = rla.compute(all_recs_joined, test_data)
results.head()

Cannot find the normalizer txt file


UnboundLocalError: local variable 'lines' referenced before assignment

In [ ]:
results.groupby('Algorithm').rND.mean()

In [ ]:
results.groupby('Algorithm').rND.mean().plot.bar()